In [1]:
import pandas as pd 
import datetime
from datetime import datetime 
import os
import warnings

In [2]:
warnings.filterwarnings('ignore')

#### ส่วนจัดการไฟล์

In [3]:
#เปลี่ยน path
path = r'C:\Users\charoenchai.chu\Desktop\Rules\สาขาแม่ฮ่องสอน และสาขาแม่สะเรียง'
file_name00 = 'ORD_422'

file_path00 = os.path.join(path, file_name00 + '.xlsx')
df=pd.read_excel(file_path00 , skiprows= 4)
df['รหัสสาขา'] = file_name00[-3:]

In [6]:
# ตั้งชื่อ field ใหม่ทั้งหมด
new_column_names = ['ที่', 'กรมธรรม์', 'ชื่อ-สกุล', 'ที่อยู่1', 'วันที่จ่ายเงิน', 'ปี','จำนวนเงิน' ,'รับเอง' ,'รับมอบฯ' ,'null','รหัสสาขา']
df.columns = new_column_names
# แทนที่ค่า null ด้วยตัวก่อนหน้า
# df.loc[df['กรมธรรม์'].str.len() < 6, 'กรมธรรม์'] = None
# df['กรมธรรม์'] = df['กรมธรรม์'].fillna(method='ffill')
df['กรมธรรม์'] = df['กรมธรรม์'].astype(str)
length = df['กรมธรรม์'].str.len()
df['กรมธรรม์'] = df['กรมธรรม์'].where(length >= 6, df['กรมธรรม์'].shift())
# เปลี่ยน type ข้อมูล
dtypes = {
        "ที่": "str",
        "กรมธรรม์": "str",
        "รหัสสาขา": "str",
        "ชื่อ-สกุล": "object",
        "ที่อยู่1": "object", 
        # "ที่อยู่2": "object",
        "วันที่จ่ายเงิน": "object",
        "ปี": "object",
        "จำนวนเงิน": "str",
        "รับเอง": "object",
        "รับมอบฯ": "object"
    }
df = df.astype(dtypes)
# ตัดแถวว่างออก
df = df[df['ที่อยู่1'] . notnull()]
# df.head()

In [7]:
# เอาที่เลขกรมธรรม์เป็น null ออก เอาไว้ไปเติมข้อมูล
null_df = df[df['วันที่จ่ายเงิน'] . isnull()]
null_df.head()
null_field = ['กรมธรรม์' , 'ที่อยู่1']
null_df = null_df[null_field]
# เปลี่ยนชื่อ field
null_df.columns = ['กรมธรรม์' , 'ที่อยู่2']
# null_df.head()

In [8]:
# ดึงเลขกรมธรรม์ที่ไม่ null
notnull_df = df[df['วันที่จ่ายเงิน'] .notnull()]
# notnull_df.head()

In [9]:
# ่join data
data_df = pd.merge(notnull_df, null_df, on='กรมธรรม์', how='left')

# เรียงลำดับคอลัมน์ใหม่ 
column_order = ['ที่', 'กรมธรรม์','รหัสสาขา' , 'ชื่อ-สกุล', 'ที่อยู่1', 'ที่อยู่2', 'วันที่จ่ายเงิน', 'ปี','จำนวนเงิน' ,'รับเอง' ,'รับมอบฯ' ,'null']
data_df = data_df[column_order]

# ตัด culumn null ออก
data_df = data_df[['ที่', 'กรมธรรม์','รหัสสาขา' , 'ชื่อ-สกุล', 'ที่อยู่1', 'ที่อยู่2', 'วันที่จ่ายเงิน', 'ปี','จำนวนเงิน' ,'รับเอง' ,'รับมอบฯ']]
# data_df.head()

In [10]:
# ตัดแถวที่ไม่ต้องการ
data_df_finish = data_df[data_df['ปี'] .notnull()]

In [11]:
# data_df_finish.to_excel(os.path.join(path, file_name00 + '&' + file_name01 + '_finish' + '.xlsx'),index = False)

#### ส่วน ACL

##### จัดไฟล์ตาม ACL

In [12]:
# เอาไว้อ่าน name
#เปลี่ยน path
path_name = r'C:\Users\charoenchai.chu\Desktop\Rules'
file_name_name = 'Name_test_DeleteField'

name_file = os.path.join(path_name, file_name_name + '.txt')
df_name = pd.read_csv(name_file , delimiter='%' , header=None, names=['nameid', 'personid'])
df_name['nameid']= df_name['nameid'].astype(str).str.zfill(13)
df_name['personid'] = df_name['personid'].astype('str').str.zfill(13)
# df_name

In [13]:
# เอาไว้อ่าน ordmast
#เปลี่ยน path
path_ord = r'C:\Users\charoenchai.chu\Desktop\Rules'
file_ord = 'ordmast_tast_DeleteField'

ord_file = os.path.join(path_ord, file_ord + '.txt')
df_ord_ORG = pd.read_csv(ord_file , delimiter= '%' , header=None, names=['กรมธรรม์','nameid', 'branch','effectivedate', 'duedate', 'addressid'])

df_ord = df_ord_ORG[['กรมธรรม์','nameid', 'branch', 'duedate', 'addressid',]]

df_ord['กรมธรรม์']= df_ord['กรมธรรม์'].astype(str).str.zfill(8)
df_ord['nameid']= df_ord['nameid'].astype(str).str.zfill(13)
df_ord['branch'] = df_ord['branch'].astype(str).str.zfill(3)
df_ord['addressid'] = df_ord['addressid'].astype(str).str.zfill(10)
# df_ord


In [14]:
# เอาไว้อ่าน person
#เปลี่ยน path
path_person = r'C:\Users\charoenchai.chu\Desktop\Rules'
file_person = 'person_test_deleatfield'

person_file = os.path.join(path_person, file_person + '.txt')
df_person = pd.read_csv(person_file ,encoding='Windows-1252' , delimiter ='%' , header=None, names=['personid', 'referenceid'])
df_person['personid'] = df_person['personid'].astype(str).str.zfill(13)
df_person['referenceid'] = df_person['referenceid'].astype(str).str.zfill(13)
# df_person

In [15]:
# เอาไว้อ่าน person
#เปลี่ยน path
path_phon = r'C:\Users\charoenchai.chu\Desktop\Rules'
file_phon = 'phon_test_deletefield'

phon_file = os.path.join(path_phon, file_phon + '.txt')
df_phon = pd.read_csv(phon_file ,encoding='Windows-1252' , delimiter ='%' , header=None, names=['addressid', 'phoneno'])
df_phon['addressid'] = df_phon['addressid'].astype(str).str.zfill(10)
df_phon['phoneno'] = df_phon['phoneno'].astype(str).str.zfill(10)
# df_phon

In [16]:
file_df = pd.merge(data_df_finish,df_ord , on= 'กรมธรรม์' , how='left')
file_df1 = pd.merge(file_df ,df_name , on= 'nameid' , how='left')
file_df2 = pd.merge(file_df1 ,df_person , on= 'personid' , how='left')
file_df3 = pd.merge(file_df2 ,df_phon , on= 'addressid' , how='left')
# file_df3.head()

In [17]:
acl_df = file_df3[['ที่','กรมธรรม์','รหัสสาขา','ชื่อ-สกุล', 'ที่อยู่1','ที่อยู่2', 'วันที่จ่ายเงิน','ปี','จำนวนเงิน','รับเอง', 'รับมอบฯ','duedate','personid','referenceid','phoneno']]

acl_df['จำนวนเงิน'] = acl_df['จำนวนเงิน'].astype(str).str.replace(',', '').astype(float)

dtypes ={"duedate" : "object"}
acl_df = acl_df.astype(dtypes)
# acl_df.head(1)

In [18]:
# ฟังก์ชันแปลงวันที่จาก พ.ศ. เป็น ค.ศ.
def convert_date(date_str):
    year = int(date_str[:4]) - 543  # แปลงปีจาก พ.ศ. เป็น ค.ศ.
    month = date_str[4:6]
    day = date_str[6:8]
    return f"{year}/{month}/{day}"

acl_df['duedate'] = acl_df['duedate'].astype(str)
# สร้างคอลัมน์ใหม่ใน DataFrame
acl_df['duedate_CTOD'] = acl_df['duedate'].apply(convert_date)
# acl_df.head()

In [19]:
# ฟังก์ชันแปลงวันที่
def convert_date(date_str):
    if pd.notna(date_str):
        try:
            day, month, year = map(int, date_str.split('/'))
            year -= 543  # ปรับปีพุทธศักราชเป็นคริสต์ศักราช
            return f"{year:04d}/{month:02d}/{day:02d}"
        except ValueError:
            return None  # คืนค่า None หากไม่สามารถแปลงได้
    return None

# สร้างคอลัมน์ใหม่ใน DataFrame
acl_df['วันที่จ่ายเงิน_CTOD'] = acl_df['วันที่จ่ายเงิน'].apply(convert_date)
# acl_df.head(1)

In [20]:
# Convert date format ให้เป็น datetime format
acl_df['วันที่จ่ายเงิน_CTOD'] = pd.to_datetime(acl_df['วันที่จ่ายเงิน_CTOD'], format='%Y/%m/%d')
acl_df['duedate_CTOD'] = pd.to_datetime(acl_df['duedate_CTOD'], format='%Y/%m/%d')

# หาความต่างของวันที่
acl_df['review_ขาดการชำระเบี้ย'] = (acl_df['วันที่จ่ายเงิน_CTOD'] - acl_df['duedate_CTOD']).dt.days
acl_df = acl_df.drop_duplicates()
# acl_df

##### Rules2 ORD

In [21]:
# (review_ขาดการชำระเบี้ย > 365) AND (รับมอบฯ = "/")
Rules_ORD_201 = acl_df[(acl_df['review_ขาดการชำระเบี้ย'] > 365) & 
                         (acl_df['รับมอบฯ'] == "/")]
Rules_ORD_201['Rules']= 1
Rules_ORD_201['Remark'] = 'รับมอบ_ขาดชำระเกิน1ปี'
# Rules_ORD_201

In [22]:
# (((review_ขาดการชำระเบี้ย >= 0) AND (review_ขาดการชำระเบี้ย <= 365))) AND (รับมอบฯ = "/")
Rules_ORD_202 = acl_df[(acl_df['review_ขาดการชำระเบี้ย'] <= 365) &
                         (acl_df['review_ขาดการชำระเบี้ย'] >  0 ) &
                         (acl_df['รับมอบฯ'] == "/")]
Rules_ORD_202['Rules']= 2
Rules_ORD_202['Remark'] = 'รับมอบ_ขาดชำระไม่เกิน1ปี'
# Rules_ORD_202.head()

In [23]:
#  (review_ขาดการชำระเบี้ย > 365) AND (รับเอง = "/")
Rules_ORD_203 = acl_df[(acl_df['review_ขาดการชำระเบี้ย'] > 365) &
                         (acl_df['รับเอง'] == "/")]
Rules_ORD_203['Rules']= 3
Rules_ORD_203['Remark'] = 'รับเอง_ขาดชำระเกิน1ปี'
# Rules_ORD_203.head()

In [24]:
# (((review_ขาดการชำระเบี้ย >= 0) AND (review_ขาดการชำระเบี้ย <= 365))) AND (รับเอง = "/")
Rules_ORD_204 = acl_df[(acl_df['review_ขาดการชำระเบี้ย'] <= 365) &
                         (acl_df['review_ขาดการชำระเบี้ย'] >  0 ) &
                         (acl_df['รับเอง'] == "/")]
Rules_ORD_204['Rules']= 4
Rules_ORD_204['Remark'] = 'รับเอง_ขาดชำระไม่เกิน1ปี'
# Rules_ORD_204.head()

In [25]:
# รวมไฟล์
Rules2_ORD_df = pd.concat([Rules_ORD_201, Rules_ORD_202 ,Rules_ORD_203 ,Rules_ORD_204], ignore_index=True)

In [26]:
Rules2_ORD_df['phoneno'] = Rules2_ORD_df['phoneno'].astype(str).str.replace('.0', '')
if len(Rules2_ORD_df['phoneno']) >= 8 :
       Rules2_ORD_df['phoneno'] = Rules2_ORD_df['phoneno'].apply(lambda x: '0' + x if len(x) > 7 else x)
Rules2_ORD_df['phoneno'] = Rules2_ORD_df['phoneno'].astype(str).str.replace('nan', '')
Rules2_ORD_df['referenceid'] = Rules2_ORD_df['referenceid'].astype(str)
Rules2_ORD_df['personid'] = Rules2_ORD_df['personid'].astype(str)
# Rules2_ORD_df['duedate'] = pd.to_datetime(Rules2_ORD_df['duedate'], format='%d/%m/%Y')
Rules2_ORD_df = Rules2_ORD_df.sort_values(by=['Rules', 'จำนวนเงิน'], ascending=[True, False])
# Rules2_ORD_df.head(2)

In [27]:
now = datetime.now().strftime("%d%m%Y")

# แยก DataFrame ตาม Remark และจัดเรียงตามขนาดกลุ่ม
grouped = Rules2_ORD_df.groupby(['Remark', 'Rules', 'รหัสสาขา'])
sorted_groups = sorted(grouped, key=lambda x: len(x[1]))

# export แบ่งไฟล์ตาม Remark
for (remark, rules, branch_code), group in sorted_groups:
    record_count = group.shape[0]  # นับจำนวน record หลังการแบ่งไฟล์
    
    file_name = f"{rules}_OPsurrenderORD_{branch_code}_{remark}_{now}_{record_count}.xlsx"
    file_path = os.path.join(path, file_name)
    
    # ส่งออก DataFrame เป็นไฟล์ Excel
    group.to_excel(file_path, index=False)

##### ส่วนจาก Rule ข้อ 11 เป็นต้นไป

In [28]:
# ฟังก์ชันแปลงวันที่จาก พ.ศ. เป็น ค.ศ.
def convert_date(date_str):
    year = int(date_str[:4]) - 543  # แปลงปีจาก พ.ศ. เป็น ค.ศ.
    month = date_str[4:6]
    day = date_str[6:8]
    return f"{year}/{month}/{day}"

df_ord_ORG['effectivedate'] = df_ord_ORG['effectivedate'].astype(str)
# สร้างคอลัมน์ใหม่ใน DataFrame
df_ord_ORG['effectivedate_CTOD'] = df_ord_ORG['effectivedate'].apply(convert_date)

In [29]:
now = datetime.now().strftime("%d%m%Y")

# กำหนดวันที่เริ่มต้น
start_date = '2022-01-01'

# แปลงคอลัมน์วันที่ให้เป็น datetime
df_ord_ORG['effectivedate_CTOD'] = pd.to_datetime(df_ord_ORG['effectivedate_CTOD'])

# กรองข้อมูลตามช่วงวันที่
df_ord_30day = df_ord_ORG[df_ord_ORG['effectivedate_CTOD'] >= start_date]
# df_ord_30day

In [30]:
df_ord_30day['nameid']= df_ord_30day['nameid'].astype(str).str.zfill(13)
df_ord_30day01 = pd.merge (df_ord_30day, df_name , on= 'nameid', how= 'left')

df_ord_30day01['personid'] = df_ord_30day01['personid'].astype('str').str.zfill(13)
df_ord_30day02 = pd.merge (df_ord_30day01,df_person , on= 'personid' , how= 'left')
df_ord_30day02 = df_ord_30day02[['effectivedate_CTOD','referenceid']]
# df_ord_30day02

In [31]:
# join กับ data table หลัก
df_ord_30day02['referenceid'] = df_ord_30day02['referenceid'].astype(str).str.zfill(13)
df_ord_30day03 = pd.merge (df_ord_30day02,acl_df , on= 'referenceid' , how='left')
# df_ord_30day03

In [32]:
df_30day_T = df_ord_30day03[df_ord_30day03['ที่'].notnull()]
# df_30day_T.head()

In [33]:
# Convert date format ให้เป็น datetime format
df_30day_T['วันที่จ่ายเงิน_CTOD'] = pd.to_datetime(df_30day_T['วันที่จ่ายเงิน_CTOD'], format='%Y/%m/%d')
df_30day_T['effectivedate_CTOD'] = pd.to_datetime(df_30day_T['effectivedate_CTOD'], format='%Y/%m/%d')

# หาความต่างของวันที่
df_30day_T['review_เคสใหม่'] = (df_30day_T['วันที่จ่ายเงิน_CTOD'] - df_30day_T['effectivedate_CTOD']).dt.days
df_30day_T = df_30day_T.drop_duplicates()
df_30day_T = df_30day_T[(df_30day_T['review_เคสใหม่'] <= 30) &
                        (df_30day_T['review_เคสใหม่'] >= -30)]
df_30day_T

,effectivedate_CTOD,referenceid,ที่,กรมธรรม์,รหัสสาขา,ชื่อ-สกุล,ที่อยู่1,ที่อยู่2,วันที่จ่ายเงิน,ปี,จำนวนเงิน,รับเอง,รับมอบฯ,duedate,personid,phoneno,duedate_CTOD,วันที่จ่ายเงิน_CTOD,review_ขาดการชำระเบี้ย,review_เคสใหม่
60916,2023-10-20,5580600079640,2,21817311,422,นายสถาพร สิริมาลัย,15/11 หมู่ที่ 1 หมู่บ้านจอมกิตติ,ต.แม่ยวม อ.แม่สะเรียง จ.แม่ฮ่องสอน 58110,04/10/2566,14/10,63100.0,/,NaN,25660924,9000000141477,0869247515,2023-09-24,2023-10-04,10.0,-16
412839,2023-09-30,1580400169570,1,23203755,422,นายอชิระ ขวัญคำ,ื81/3 หมู่ที่ 3 ซอย3 ถนนหลวง108,ต.แม่สะเรียง อ.แม่สะเรียง จ.แม่ฮ่องสอน 58110,25/09/2566,04/05,3817.0,/,NaN,25660830,9000095382880,0623097849,2023-08-30,2023-09-25,26.0,-5
412840,2023-09-30,1580400169570,1,23203755,422,นายอชิระ ขวัญคำ,ื81/3 หมู่ที่ 3 ซอย3 ถนนหลวง108,ต.แม่สะเรียง อ.แม่สะเรียง จ.แม่ฮ่องสอน 58110,25/09/2566,04/05,3817.0,/,NaN,25660830,9000095382880,0636565686,2023-08-30,2023-09-25,26.0,-5
412841,2023-09-30,1580400169570,1,23203755,422,นายอชิระ ขวัญคำ,ื81/3 หมู่ที่ 3 ซอย3 ถนนหลวง108,ต.แม่สะเรียง อ.แม่สะเรียง จ.แม่ฮ่องสอน 58110,25/09/2566,04/05,3817.0,/,NaN,25660830,9000095382880,0632544232,2023-08-30,2023-09-25,26.0,-5


##### Rules 30 วัน

In [34]:
# (review_ขาดการชำระเบี้ย > 365) AND (รับมอบฯ = "/")
ORD_30day_201 = df_30day_T[(df_30day_T['review_ขาดการชำระเบี้ย'] > 365) & 
                         (df_30day_T['รับมอบฯ'] == "/")]
ORD_30day_201['Rules']= 1
ORD_30day_201['Remark'] = 'รับมอบ_ขาดชำระเกิน1ปี'
# Rules_ORD_201

In [35]:
# (((review_ขาดการชำระเบี้ย >= 0) AND (review_ขาดการชำระเบี้ย <= 365))) AND (รับมอบฯ = "/")
ORD_30day_202 = df_30day_T[(df_30day_T['review_ขาดการชำระเบี้ย'] <= 365) &
                         (df_30day_T['review_ขาดการชำระเบี้ย'] >  0 ) &
                         (df_30day_T['รับมอบฯ'] == "/")]
ORD_30day_202['Rules']= 2
ORD_30day_202['Remark'] = 'รับมอบ_ขาดชำระไม่เกิน1ปี'
# Rules_ORD_202.head()

In [36]:
#  (review_ขาดการชำระเบี้ย > 365) AND (รับเอง = "/")
ORD_30day_203 = df_30day_T[(df_30day_T['review_ขาดการชำระเบี้ย'] > 365) &
                         (df_30day_T['รับเอง'] == "/")]
ORD_30day_203['Rules']= 3
ORD_30day_203['Remark'] = 'รับเอง_ขาดชำระเกิน1ปี'
# Rules_ORD_203.head()

In [37]:
# (((review_ขาดการชำระเบี้ย >= 0) AND (review_ขาดการชำระเบี้ย <= 365))) AND (รับเอง = "/")
ORD_30day_204 = df_30day_T[(df_30day_T['review_ขาดการชำระเบี้ย'] <= 365) &
                         (df_30day_T['review_ขาดการชำระเบี้ย'] >  0 ) &
                         (df_30day_T['รับเอง'] == "/")]
ORD_30day_204['Rules']= 4
ORD_30day_204['Remark'] = 'รับเอง_ขาดชำระไม่เกิน1ปี'
# ORD_30day_204.head()

In [38]:
# รวมไฟล์
ORD_30day_df = pd.concat([ORD_30day_201, ORD_30day_202 ,ORD_30day_203 ,ORD_30day_204], ignore_index=True)
ORD_30day_df

,effectivedate_CTOD,referenceid,ที่,กรมธรรม์,รหัสสาขา,ชื่อ-สกุล,ที่อยู่1,ที่อยู่2,วันที่จ่ายเงิน,ปี,...,รับมอบฯ,duedate,personid,phoneno,duedate_CTOD,วันที่จ่ายเงิน_CTOD,review_ขาดการชำระเบี้ย,review_เคสใหม่,Rules,Remark
0,2023-10-20,5580600079640,2,21817311,422,นายสถาพร สิริมาลัย,15/11 หมู่ที่ 1 หมู่บ้านจอมกิตติ,ต.แม่ยวม อ.แม่สะเรียง จ.แม่ฮ่องสอน 58110,04/10/2566,14/10,...,NaN,25660924,9000000141477,0869247515,2023-09-24,2023-10-04,10.0,-16,4,รับเอง_ขาดชำระไม่เกิน1ปี
1,2023-09-30,1580400169570,1,23203755,422,นายอชิระ ขวัญคำ,ื81/3 หมู่ที่ 3 ซอย3 ถนนหลวง108,ต.แม่สะเรียง อ.แม่สะเรียง จ.แม่ฮ่องสอน 58110,25/09/2566,04/05,...,NaN,25660830,9000095382880,0623097849,2023-08-30,2023-09-25,26.0,-5,4,รับเอง_ขาดชำระไม่เกิน1ปี
2,2023-09-30,1580400169570,1,23203755,422,นายอชิระ ขวัญคำ,ื81/3 หมู่ที่ 3 ซอย3 ถนนหลวง108,ต.แม่สะเรียง อ.แม่สะเรียง จ.แม่ฮ่องสอน 58110,25/09/2566,04/05,...,NaN,25660830,9000095382880,0636565686,2023-08-30,2023-09-25,26.0,-5,4,รับเอง_ขาดชำระไม่เกิน1ปี
3,2023-09-30,1580400169570,1,23203755,422,นายอชิระ ขวัญคำ,ื81/3 หมู่ที่ 3 ซอย3 ถนนหลวง108,ต.แม่สะเรียง อ.แม่สะเรียง จ.แม่ฮ่องสอน 58110,25/09/2566,04/05,...,NaN,25660830,9000095382880,0632544232,2023-08-30,2023-09-25,26.0,-5,4,รับเอง_ขาดชำระไม่เกิน1ปี


In [40]:
now = datetime.now().strftime("%d%m%Y")

# แยก DataFrame ตาม Remark และจัดเรียงตามขนาดกลุ่ม
grouped = ORD_30day_df.groupby(['Remark', 'Rules', 'รหัสสาขา'])
sorted_groups = sorted(grouped, key=lambda x: len(x[1]))

# export แบ่งไฟล์ตาม Remark
for (remark, rules, branch_code), group in sorted_groups:
    record_count = group.shape[0]  # นับจำนวน record หลังการแบ่งไฟล์
    
    file_name = f"{rules}_30day_OPsurrenderORD_{branch_code}_{remark}_{now}_{record_count}.xlsx"
    file_path = os.path.join(path, file_name)
    
    # ส่งออก DataFrame เป็นไฟล์ Excel
    group.to_excel(file_path, index=False)